In [1]:
import json
import pandas as pd

In [3]:
books_file = "data/goodreads_books_comics_graphic.json.gz"
goodreads = pd.read_json(books_file, lines=True, compression="gzip")
goodreads.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,,1,[],US,,"[{'count': '228', 'name': 'to-read'}, {'count'...",B00NLXQ534,true,4.12,,...,,,,https://www.goodreads.com/book/show/25742454-t...,https://s.gr-assets.com/assets/nophoto/book/11...,25742454,1,42749946,The Switchblade Mamma,The Switchblade Mamma
1,2205073346,2,[],US,fre,"[{'count': '2', 'name': 'bd'}, {'count': '2', ...",,false,3.94,,...,1,,2016,https://www.goodreads.com/book/show/30128855-c...,https://images.gr-assets.com/books/1462644346m...,30128855,16,50558228,Cruelle,Cruelle
2,,5,"[246830, 362583, 362581, 623032]",US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",,false,4.28,,...,,,2012,https://www.goodreads.com/book/show/13571772-c...,https://images.gr-assets.com/books/1333287305m...,13571772,51,102217,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...
3,,1,[],US,eng,"[{'count': '222', 'name': 'to-read'}, {'count'...",B06XKGGSB7,true,4.05,B06XKGGSB7,...,,,,https://www.goodreads.com/book/show/35452242-b...,https://s.gr-assets.com/assets/nophoto/book/11...,35452242,6,54276229,Bounty Hunter 4/3: My Life in Combat from Mari...,Bounty Hunter 4/3: My Life in Combat from Mari...
4,0930289765,6,"[266759, 1096220]",US,en-US,"[{'count': '20', 'name': 'to-read'}, {'count':...",,false,4.06,,...,11,,1997,https://www.goodreads.com/book/show/707611.Sup...,https://images.gr-assets.com/books/1307838888m...,707611,51,693886,"Superman Archives, Vol. 2","Superman Archives, Vol. 2"


In [4]:
goodreads['author_ids'] = goodreads['authors'].apply(lambda x: {int(author['author_id']) for author in x})

In [5]:
goodreads['series_ids'] = goodreads['series'].apply(lambda x: {int(s) for s in x})

In [6]:
cols_to_keep = goodreads.columns

In [7]:
import numpy as np

# Define cols_from_max_rating as the list of columns to select (using cols_to_keep)
cols_from_max_rating = cols_to_keep

# Group by work_id and keep only the specified columns

# First, sort the dataframe by work_id and ratings_count (descending)
sorted_df = goodreads.sort_values(['work_id', 'ratings_count'], ascending=[True, False])

# Define a function to get the best values from a group of books with the same work_id
def get_best_row(group):
    # Start with the row with the highest ratings_count
    best_row = group.iloc[0].copy()
    
    # For each column we want to potentially fill from other books
    for col in ['description', 'publication_year', 'num_pages']:
        # If the value is empty
        if pd.isna(best_row[col]) or best_row[col] == '':
            # Look for a non-empty value in the group
            non_empty = group[(~group[col].isna()) & (group[col] != '')]
            if len(non_empty) > 0:
                best_row[col] = non_empty.iloc[0][col]
    
    return best_row

# Apply the function to each group of books with the same work_id
best_rows = []
for _, group in sorted_df.groupby('work_id'):
    best_rows.append(get_best_row(group))

# Create a dataframe from the best rows
max_ratings_data = pd.DataFrame(best_rows)[cols_from_max_rating]

# Calculate the aggregated statistics
agg_stats = goodreads.groupby('work_id').agg({
    'ratings_count': 'sum',
    'average_rating': 'mean'
}).reset_index()

# Combine the data using suffixes so that the aggregated columns keep the names we need
goodreads_works = max_ratings_data.merge(agg_stats, on='work_id', suffixes=('_orig', ''))

# Ensure the columns are in the right order
goodreads_works = goodreads_works[cols_to_keep]

# Display the shape of the resulting dataframe
print(f"Original dataframe: {goodreads.shape}")
print(f"Aggregated dataframe: {goodreads_works.shape}")

Original dataframe: (89411, 31)
Aggregated dataframe: (62944, 31)


In [8]:
description_criterion = goodreads_works[goodreads_works['description'].apply(lambda x: len(x) != 0)]['work_id']

In [9]:
# Filtering for english books (mainly)

#goodreads_children['language_code'].apply(lambda x: "" if re.match()) # Change any number of spaces to just empty string
language_criterion = goodreads[goodreads['language_code'].apply(lambda x: x in ["", 'eng', 'en-US', 'en-GB', 'en-CA'])]['work_id']

In [10]:
final = goodreads_works[goodreads_works['work_id'].isin(description_criterion) & goodreads_works['work_id'].isin(language_criterion)]

In [11]:
final['num_pages'] = final['num_pages'].apply(lambda x: int(x) if len(x) > 0 else '')
final['publication_year'] = final['publication_year'].apply(lambda x: int(x) if len(x) > 0 else '')
final['publication_month'] = final['publication_month'].apply(lambda x: int(x) if len(x) > 0 else '')
final['publication_day'] = final['publication_day'].apply(lambda x: int(x) if len(x) > 0 else '')

C:\Users\rynoc\AppData\Local\Temp\ipykernel_1996\642797579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['num_pages'] = final['num_pages'].apply(lambda x: int(x) if len(x) > 0 else '')
C:\Users\rynoc\AppData\Local\Temp\ipykernel_1996\642797579.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['publication_year'] = final['publication_year'].apply(lambda x: int(x) if len(x) > 0 else '')
C:\Users\rynoc\AppData\Local\Temp\ipykernel_1996\642797579.py:3: SettingWithCopyWarning: 
A value is try

In [12]:
final['is_ebook'] = final['is_ebook'].apply(lambda x: True if x == 'true' else False)

C:\Users\rynoc\AppData\Local\Temp\ipykernel_1996\222911999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['is_ebook'] = final['is_ebook'].apply(lambda x: True if x == 'true' else False)


In [13]:
final = final.replace('', None)
def assemble_date(row):
    # Check for missing year; if missing, return None (which maps to SQL NULL)
    year_val = row.get('publication_year')
    if not year_val:
        return None
    try:
        year = year_val
    except ValueError:
        return None

    # For month, default to 1 if missing
    month_val = row.get('publication_month')
    if not month_val:
        month = 1
    else:
        try:
            month = month_val
        except ValueError:
            month = 1

    # For day, default to 1 if missing
    day_val = row.get('publication_day')
    if not day_val:
        day = 1
    else:
        try:
            day = day_val
        except ValueError:
            day = 1

    # Create the date object; if the values are invalid, it will raise an error
    try:
        return pd.Timestamp(year, month, day)
    except Exception:
        return None

# Apply the function row-wise to create the publication_date column
final['publication_date'] = final.apply(assemble_date, axis=1)

In [14]:
final

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,url,image_url,book_id,ratings_count,work_id,title,title_without_series,author_ids,series_ids,publication_date
0,1563898586,1152,"[185613, 458201]",US,eng,"[{'count': '2020', 'name': 'graphic-novels'}, ...",None,False,3.93,B0064W63XG,...,https://www.goodreads.com/book/show/297627.The...,https://images.gr-assets.com/books/1327894724m...,297627,42046,797,"The League of Extraordinary Gentlemen, Vol. 1","The League of Extraordinary Gentlemen, Vol. 1","{3961, 61919}","{458201, 185613}",2002-10-01 00:00:00
1,1887279806,108,[248363],US,None,"[{'count': '1438', 'name': 'to-read'}, {'count...",None,False,4.19,None,...,https://www.goodreads.com/book/show/89816.Kabu...,https://s.gr-assets.com/assets/nophoto/book/11...,89816,1674,1027,"Kabuki, Vol. 1: Circle of Blood","Kabuki, Vol. 1: Circle of Blood",{10455},{248363},2010-03-30 00:00:00
2,0224063979,879,[],US,None,"[{'count': '13038', 'name': 'to-read'}, {'coun...",None,False,4.10,None,...,https://www.goodreads.com/book/show/34072.Jimm...,https://images.gr-assets.com/books/1327962921m...,34072,19221,1315,"Jimmy Corrigan, the Smartest Kid on Earth","Jimmy Corrigan, the Smartest Kid on Earth",{5112},{},2003-05-22 00:00:00
3,0375422951,67,[309090],US,en-US,"[{'count': '921', 'name': 'to-read'}, {'count'...",None,False,4.35,None,...,https://www.goodreads.com/book/show/7552.The_A...,https://images.gr-assets.com/books/1415413456m...,7552,2240,1398,The Acme Novelty Library (Pantheon Graphic Nov...,The Acme Novelty Library (Pantheon Graphic Nov...,{5112},{309090},None
4,1563892049,649,[144460],US,eng,"[{'count': '12039', 'name': 'to-read'}, {'coun...",None,False,4.59,B0064W64UI,...,https://www.goodreads.com/book/show/71252.The_...,https://images.gr-assets.com/books/1339922887m...,71252,36473,2647,The Kindly Ones (The Sandman #9),The Kindly Ones (The Sandman #9),"{1221698, 40292, 14022, 321799, 10763, 779, 28...",{144460},1999-08-06 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62928,0785190376,13,[1135894],US,eng,"[{'count': '9', 'name': 'to-read'}, {'count': ...",None,False,3.67,B01EZ6RPUW,...,https://www.goodreads.com/book/show/18948781-d...,https://images.gr-assets.com/books/1404483758m...,18948781,42,58256583,Devil Dinosaur by Jack Kirby: The Complete Col...,Devil Dinosaur by Jack Kirby: The Complete Col...,"{133284, 150001, 44724, 795637, 10299}",{1135894},2014-06-17 00:00:00
62932,1628751886,7,[],US,eng,"[{'count': '16', 'name': 'to-read'}, {'count':...",None,False,4.07,B076FLNRX1,...,https://www.goodreads.com/book/show/36260018-4...,https://s.gr-assets.com/assets/nophoto/book/11...,36260018,15,58282808,4 Kids Walk Into A Bank TP,4 Kids Walk Into A Bank TP,"{14254218, 388934}",{},None
62934,None,3,[1138023],US,None,"[{'count': '1', 'name': 'wish-list'}, {'count'...",None,False,3.93,None,...,https://www.goodreads.com/book/show/19659208-s...,https://s.gr-assets.com/assets/nophoto/book/11...,19659208,19,58292587,Spider-Man Ultimate Collection,Spider-Man Ultimate Collection,"{12736, 6542722, 18160}",{1138023},None
62938,0785110089,9,[],US,None,"[{'count': '27', 'name': 'to-read'}, {'count':...",None,False,3.77,B00ZMP23Y4,...,https://www.goodreads.com/book/show/657991.Ult...,https://s.gr-assets.com/assets/nophoto/book/11...,657991,130,58292642,"Ultimate X-Men, Vol. 1 (Ultimate X-Men hardcov...","Ultimate X-Men, Vol. 1 (Ultimate X-Men hardcov...",{12736},{},2002-08-01 00:00:00


In [15]:
final = final[['book_id','work_id','isbn','isbn13','asin','kindle_asin','series_ids','format','publisher', 
      'author_ids', 'link','publication_date', 'publication_year', 'publication_month', 
      'publication_day', 'title', 'title_without_series', 'is_ebook', 'average_rating',
     'text_reviews_count', 'ratings_count', 'description', 'num_pages']]
final

,book_id,work_id,isbn,isbn13,asin,kindle_asin,series_ids,format,publisher,author_ids,...,publication_month,publication_day,title,title_without_series,is_ebook,average_rating,text_reviews_count,ratings_count,description,num_pages
0,297627,797,1563898586,9781563898587,None,B0064W63XG,"{458201, 185613}",Paperback,America's Best Comics,"{3961, 61919}",...,10,1,"The League of Extraordinary Gentlemen, Vol. 1","The League of Extraordinary Gentlemen, Vol. 1",False,3.93,1152,42046,"London, 1898. The Victorian Era draws to a clo...",192
1,89816,1027,1887279806,9781887279802,None,None,{248363},Paperback,Image Comics,{10455},...,3,30,"Kabuki, Vol. 1: Circle of Blood","Kabuki, Vol. 1: Circle of Blood",False,4.19,108,1674,Collecting all six issues of the first Kabuki ...,272
2,34072,1315,0224063979,9780224063975,None,None,{},Paperback,Jonathan Cape,{5112},...,5,22,"Jimmy Corrigan, the Smartest Kid on Earth","Jimmy Corrigan, the Smartest Kid on Earth",False,4.10,879,19221,Jimmy Corriganhas rightly been hailed as the g...,380
3,7552,1398,0375422951,9780375422959,None,None,{309090},None,None,{5112},...,None,None,The Acme Novelty Library (Pantheon Graphic Nov...,The Acme Novelty Library (Pantheon Graphic Nov...,False,4.35,67,2240,Utterly eschewing the general bonhomie surroun...,None
4,71252,2647,1563892049,9781563892042,None,B0064W64UI,{144460},Hardcover,Vertigo,"{1221698, 40292, 14022, 321799, 10763, 779, 28...",...,8,6,The Kindly Ones (The Sandman #9),The Kindly Ones (The Sandman #9),False,4.59,649,36473,The Penultimate volume to the phenomenal Sandm...,352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62928,18948781,58256583,0785190376,9780785190370,None,B01EZ6RPUW,{1135894},Paperback,Marvel,"{133284, 150001, 44724, 795637, 10299}",...,6,17,Devil Dinosaur by Jack Kirby: The Complete Col...,Devil Dinosaur by Jack Kirby: The Complete Col...,False,3.67,13,42,"Devil Dinosaur was one of Jack ""King"" Kirby's ...",184
62932,36260018,58282808,1628751886,9781628751888,None,B076FLNRX1,{},None,None,"{14254218, 388934}",...,None,None,4 Kids Walk Into A Bank TP,4 Kids Walk Into A Bank TP,False,4.07,7,15,4 KIDS WALK INTO A BANK is the darkly comedic ...,None
62934,19659208,58292587,None,None,None,None,{1138023},None,None,"{12736, 6542722, 18160}",...,None,None,Spider-Man Ultimate Collection,Spider-Man Ultimate Collection,False,3.93,3,19,Peter Parker's worst nightmare has come true: ...,None
62938,657991,58292642,0785110089,9780785110088,None,B00ZMP23Y4,{},Hardcover,Marvel Comics Group,{12736},...,8,1,"Ultimate X-Men, Vol. 1 (Ultimate X-Men hardcov...","Ultimate X-Men, Vol. 1 (Ultimate X-Men hardcov...",False,3.77,9,130,Le monde est au bord de la guerre. Pas une gue...,352


In [16]:
final['series_ids'] = final['series_ids'].apply(lambda x: None if len(x) == 0 else x)
final['author_ids'] = final['author_ids'].apply(lambda x: None if len(x) == 0 else x)

C:\Users\rynoc\AppData\Local\Temp\ipykernel_1996\424329773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['series_ids'] = final['series_ids'].apply(lambda x: None if len(x) == 0 else x)
C:\Users\rynoc\AppData\Local\Temp\ipykernel_1996\424329773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['author_ids'] = final['author_ids'].apply(lambda x: None if len(x) == 0 else x)


In [17]:
final

,book_id,work_id,isbn,isbn13,asin,kindle_asin,series_ids,format,publisher,author_ids,...,publication_month,publication_day,title,title_without_series,is_ebook,average_rating,text_reviews_count,ratings_count,description,num_pages
0,297627,797,1563898586,9781563898587,None,B0064W63XG,"{458201, 185613}",Paperback,America's Best Comics,"{3961, 61919}",...,10,1,"The League of Extraordinary Gentlemen, Vol. 1","The League of Extraordinary Gentlemen, Vol. 1",False,3.93,1152,42046,"London, 1898. The Victorian Era draws to a clo...",192
1,89816,1027,1887279806,9781887279802,None,None,{248363},Paperback,Image Comics,{10455},...,3,30,"Kabuki, Vol. 1: Circle of Blood","Kabuki, Vol. 1: Circle of Blood",False,4.19,108,1674,Collecting all six issues of the first Kabuki ...,272
2,34072,1315,0224063979,9780224063975,None,None,None,Paperback,Jonathan Cape,{5112},...,5,22,"Jimmy Corrigan, the Smartest Kid on Earth","Jimmy Corrigan, the Smartest Kid on Earth",False,4.10,879,19221,Jimmy Corriganhas rightly been hailed as the g...,380
3,7552,1398,0375422951,9780375422959,None,None,{309090},None,None,{5112},...,None,None,The Acme Novelty Library (Pantheon Graphic Nov...,The Acme Novelty Library (Pantheon Graphic Nov...,False,4.35,67,2240,Utterly eschewing the general bonhomie surroun...,None
4,71252,2647,1563892049,9781563892042,None,B0064W64UI,{144460},Hardcover,Vertigo,"{1221698, 40292, 14022, 321799, 10763, 779, 28...",...,8,6,The Kindly Ones (The Sandman #9),The Kindly Ones (The Sandman #9),False,4.59,649,36473,The Penultimate volume to the phenomenal Sandm...,352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62928,18948781,58256583,0785190376,9780785190370,None,B01EZ6RPUW,{1135894},Paperback,Marvel,"{133284, 150001, 44724, 795637, 10299}",...,6,17,Devil Dinosaur by Jack Kirby: The Complete Col...,Devil Dinosaur by Jack Kirby: The Complete Col...,False,3.67,13,42,"Devil Dinosaur was one of Jack ""King"" Kirby's ...",184
62932,36260018,58282808,1628751886,9781628751888,None,B076FLNRX1,None,None,None,"{14254218, 388934}",...,None,None,4 Kids Walk Into A Bank TP,4 Kids Walk Into A Bank TP,False,4.07,7,15,4 KIDS WALK INTO A BANK is the darkly comedic ...,None
62934,19659208,58292587,None,None,None,None,{1138023},None,None,"{12736, 6542722, 18160}",...,None,None,Spider-Man Ultimate Collection,Spider-Man Ultimate Collection,False,3.93,3,19,Peter Parker's worst nightmare has come true: ...,None
62938,657991,58292642,0785110089,9780785110088,None,B00ZMP23Y4,None,Hardcover,Marvel Comics Group,{12736},...,8,1,"Ultimate X-Men, Vol. 1 (Ultimate X-Men hardcov...","Ultimate X-Men, Vol. 1 (Ultimate X-Men hardcov...",False,3.77,9,130,Le monde est au bord de la guerre. Pas une gue...,352


In [29]:
final.to_csv('data/goodreads_books_cleaned.csv', index=False)

In [35]:
import pandas as pd
import requests
import os

def download_data(file):
    # Create the 'data' directory if it doesn't exist
    os.makedirs('data', exist_ok=True)
    
    # Define the full path for the file
    file_path = os.path.join('data', file)
    
    with requests.get('https://mcauleylab.ucsd.edu/public_datasets/gdrive/goodreads/' + file, stream=True) as r:
        r.raise_for_status()
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
                    
    print("Download complete:", file_path)

In [38]:
# download books
authors_file = 'goodreads_book_authors.json.gz'
download_data(authors_file)
authors = pd.read_json("data/goodreads_book_authors.json.gz", lines=True, compression="gzip")
authors.head()

Download complete: data\goodreads_book_authors.json.gz


,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [39]:
final

,book_id,work_id,isbn,isbn13,asin,kindle_asin,series_ids,format,publisher,author_ids,...,publication_month,publication_day,title,title_without_series,is_ebook,average_rating,text_reviews_count,ratings_count,description,num_pages
0,297627,797,1563898586,9781563898587,None,B0064W63XG,"{458201, 185613}",Paperback,America's Best Comics,"{3961, 61919}",...,10,1,"The League of Extraordinary Gentlemen, Vol. 1","The League of Extraordinary Gentlemen, Vol. 1",False,3.93,1152,42046,"London, 1898. The Victorian Era draws to a clo...",192
1,89816,1027,1887279806,9781887279802,None,None,{248363},Paperback,Image Comics,{10455},...,3,30,"Kabuki, Vol. 1: Circle of Blood","Kabuki, Vol. 1: Circle of Blood",False,4.19,108,1674,Collecting all six issues of the first Kabuki ...,272
2,34072,1315,0224063979,9780224063975,None,None,None,Paperback,Jonathan Cape,{5112},...,5,22,"Jimmy Corrigan, the Smartest Kid on Earth","Jimmy Corrigan, the Smartest Kid on Earth",False,4.10,879,19221,Jimmy Corriganhas rightly been hailed as the g...,380
3,7552,1398,0375422951,9780375422959,None,None,{309090},None,None,{5112},...,None,None,The Acme Novelty Library (Pantheon Graphic Nov...,The Acme Novelty Library (Pantheon Graphic Nov...,False,4.35,67,2240,Utterly eschewing the general bonhomie surroun...,None
4,71252,2647,1563892049,9781563892042,None,B0064W64UI,{144460},Hardcover,Vertigo,"{1221698, 40292, 14022, 321799, 10763, 779, 28...",...,8,6,The Kindly Ones (The Sandman #9),The Kindly Ones (The Sandman #9),False,4.59,649,36473,The Penultimate volume to the phenomenal Sandm...,352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62928,18948781,58256583,0785190376,9780785190370,None,B01EZ6RPUW,{1135894},Paperback,Marvel,"{133284, 150001, 44724, 795637, 10299}",...,6,17,Devil Dinosaur by Jack Kirby: The Complete Col...,Devil Dinosaur by Jack Kirby: The Complete Col...,False,3.67,13,42,"Devil Dinosaur was one of Jack ""King"" Kirby's ...",184
62932,36260018,58282808,1628751886,9781628751888,None,B076FLNRX1,None,None,None,"{14254218, 388934}",...,None,None,4 Kids Walk Into A Bank TP,4 Kids Walk Into A Bank TP,False,4.07,7,15,4 KIDS WALK INTO A BANK is the darkly comedic ...,None
62934,19659208,58292587,None,None,None,None,{1138023},None,None,"{12736, 6542722, 18160}",...,None,None,Spider-Man Ultimate Collection,Spider-Man Ultimate Collection,False,3.93,3,19,Peter Parker's worst nightmare has come true: ...,None
62938,657991,58292642,0785110089,9780785110088,None,B00ZMP23Y4,None,Hardcover,Marvel Comics Group,{12736},...,8,1,"Ultimate X-Men, Vol. 1 (Ultimate X-Men hardcov...","Ultimate X-Men, Vol. 1 (Ultimate X-Men hardcov...",False,3.77,9,130,Le monde est au bord de la guerre. Pas une gue...,352


In [40]:
authors

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740
...,...,...,...,...,...
829524,4.36,197551,4,Patty Furbush,11
829525,4.33,3988103,3,Jim Schlinkman,6
829526,4.00,13464507,2,Rich Jolly,18
829527,3.31,7427847,1,sr@ mwrGn,13


In [41]:
import pandas as pd

# Example: final['author_ids'] is something like [{185613, 458201}, {24363}, ...]
all_author_ids = set()

for author_set in final['author_ids']:
    all_author_ids.update(author_set)

In [42]:
# Keep only rows in authors whose author_id is in all_author_ids
authors_filtered = authors[authors['author_id'].isin(all_author_ids)]
authors_filtered

,average_rating,author_id,text_reviews_count,name,ratings_count
2,3.92,10333,5075,Barbara Hambly,122118
7,4.18,19158,486,Rachel Roberts,13677
15,3.84,3083854,9732,Tom Wolfe,210744
18,4.29,4862,3992,Randy Alcorn,56585
19,3.99,137561,12146,Jennifer L. Holm,128152
...,...,...,...,...,...
828788,3.71,15225346,2,Matthew J. McEniry,7
828789,4.00,5860498,2,Robert Moses Peaslee,20
828801,3.65,3150073,509,Derek Kim,3777
828978,4.14,427078,5,Doug Thomas,14


In [43]:
authors_filtered.dtypes

average_rating        float64
author_id               int64
text_reviews_count      int64
name                   object
ratings_count           int64
dtype: object

In [44]:
authors_filtered["name"] = authors_filtered["name"].str.replace(r"\s+", " ", regex=True).str.strip()

C:\Users\rynoc\AppData\Local\Temp\ipykernel_1996\1983981045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_filtered["name"] = authors_filtered["name"].str.replace(r"\s+", " ", regex=True).str.strip()


In [45]:
authors_filtered["name"]

2               Barbara Hambly
7               Rachel Roberts
15                   Tom Wolfe
18                Randy Alcorn
19            Jennifer L. Holm
                  ...         
828788      Matthew J. McEniry
828789    Robert Moses Peaslee
828801               Derek Kim
828978             Doug Thomas
829499              Rafa Lopez
Name: name, Length: 21619, dtype: object

In [46]:
authors_filtered.to_csv('data/goodreads_authors_cleaned.csv', index=False)

# Remove columns not necessary for Neo4j